In [1]:
from pyspark.sql import SparkSession

# Initialize Spark session
spark = SparkSession.builder.appName("MergeDataFrames").getOrCreate()

# Define the file paths
script1_path = 'USMD_Arlington.py'
#script2_path = 'WhiteRockMedicalCenter.py'
script3_path = 'Adventist_health_White_Memorial.py'
script4_path = 'UCLA_Resnick.py'
script5_path = 'BaylorScott_Irving.py'

# Create a dictionary to store the variables from the scripts
script1_globals = {}
#script2_globals = {}
script3_globals = {}
script4_globals = {}
script5_globals = {}

# Execute the scripts
with open(script1_path) as f:
    exec(f.read(), script1_globals)

#with open(script2_path) as f:
   #exec(f.read(), script2_globals)
    
with open(script3_path) as f:
    exec(f.read(), script3_globals)
    
with open(script4_path) as f:
    exec(f.read(), script4_globals)
    
with open(script5_path) as f:
    exec(f.read(), script5_globals)

# Access the DataFrames from the scripts
df1 = script1_globals['df_final']
#df2 = script2_globals['df_final']
df3 = script3_globals['df_final']
df4 = script4_globals['df_final']
df5 = script5_globals['df_final']

# Merge the DataFrames, borrowing the header from the first file and ignoring headers from other files
merged_df = df1.unionByName(df3).unionByName(df4).unionByName(df5)

# Show the merged DataFrame
merged_df.show(5)

+--------------------+------+-----------------+--------------------+----------------+-------------------+-------------------+
|         description|code|1|       payer_name|           plan_name|estimated_amount|standard_charge|max|standard_charge|min|
+--------------------+------+-----------------+--------------------+----------------+-------------------+-------------------+
|HEART TRANSPLANT ...|   001|            Aetna|               Aetna|       553176.28|          835367.16|          553176.28|
|HEART TRANSPLANT ...|   001|United Healthcare|United Healthcare...|        635848.5|          835367.16|          553176.28|
|HEART TRANSPLANT ...|   001|United Healthcare|United Healthcare...|       835367.16|          835367.16|          553176.28|
|HEART TRANSPLANT ...|   001|United Healthcare|United Healthcare...|        635848.5|          835367.16|          553176.28|
|HEART TRANSPLANT ...|   002|            Aetna|               Aetna|       333274.32|          503286.98|          333

In [2]:
merged_df.show(5)

+--------------------+------------+------------------+--------------------+----------------------+--------------------+--------------------+--------------+
| billing_description|billing_code|insurance_provider|      insurance_plan|gross_standard_charges|max_standard_charges|min_standard_charges| hospital_name|
+--------------------+------------+------------------+--------------------+----------------------+--------------------+--------------------+--------------+
|HEART TRANSPLANT ...|         001|             Aetna|               Aetna|             553176.28|           835367.16|           553176.28|USMD_Arlington|
|HEART TRANSPLANT ...|         001| United Healthcare|United Healthcare...|              635848.5|           835367.16|           553176.28|USMD_Arlington|
|HEART TRANSPLANT ...|         001| United Healthcare|United Healthcare...|             835367.16|           835367.16|           553176.28|USMD_Arlington|
|HEART TRANSPLANT ...|         001| United Healthcare|United Hea

In [3]:
# Identify unique hospitals in the merged DataFrame
unique_hospitals = merged_df.select("hospital_name").distinct()
unique_hospitals.show(10)

+--------------------+
|       hospital_name|
+--------------------+
|      USMD_Arlington|
|Adventist_health_...|
|        UCLA_Resnick|
|  BaylorScott_Irving|
+--------------------+



In [4]:
from pyspark.sql import SparkSession

# Initialize Spark session
spark = SparkSession.builder.appName("MergeDataFrames").getOrCreate()

# Assuming merged_df is already defined and available
# Register the DataFrame as a SQL temporary view
merged_df.createOrReplaceTempView("merged_table")

# SQL query to check for null values in each column grouped by hospital name
null_check_query_grouped = """
SELECT 
    hospital_name,
    SUM(CASE WHEN billing_description IS NULL THEN 1 ELSE 0 END) AS billing_description_nulls,
    SUM(CASE WHEN billing_code IS NULL THEN 1 ELSE 0 END) AS billing_code_nulls,
    SUM(CASE WHEN insurance_provider IS NULL THEN 1 ELSE 0 END) AS insurance_provider_nulls,
    SUM(CASE WHEN insurance_plan IS NULL THEN 1 ELSE 0 END) AS insurance_plan_nulls,
    SUM(CASE WHEN gross_standard_charges IS NULL THEN 1 ELSE 0 END) AS gross_standard_charges_nulls,
    SUM(CASE WHEN max_standard_charges IS NULL THEN 1 ELSE 0 END) AS max_standard_charges_nulls,
    SUM(CASE WHEN min_standard_charges IS NULL THEN 1 ELSE 0 END) AS min_standard_charges_nulls
FROM merged_table
GROUP BY hospital_name
"""

# Execute the query
null_counts_grouped = spark.sql(null_check_query_grouped)

# Show the result
null_counts_grouped.show()

+--------------------+-------------------------+------------------+------------------------+--------------------+----------------------------+--------------------------+--------------------------+
|       hospital_name|billing_description_nulls|billing_code_nulls|insurance_provider_nulls|insurance_plan_nulls|gross_standard_charges_nulls|max_standard_charges_nulls|min_standard_charges_nulls|
+--------------------+-------------------------+------------------+------------------------+--------------------+----------------------------+--------------------------+--------------------------+
|      USMD_Arlington|                        0|                 6|                       0|                   0|                         231|                         6|                         6|
|Adventist_health_...|                        0|                 0|                       0|                   0|                           0|                         0|                         0|
|        UCLA_R

In [5]:
# Assuming 'spark_df' is your Spark DataFrame
pandas_df = merged_df.toPandas()

# Now 'pandas_df' is a Pandas DataFrame
print(pandas_df.head())

                                 billing_description billing_code  \
0  HEART TRANSPLANT OR IMPLANT OF HEART ASSIST SY...          001   
1  HEART TRANSPLANT OR IMPLANT OF HEART ASSIST SY...          001   
2  HEART TRANSPLANT OR IMPLANT OF HEART ASSIST SY...          001   
3  HEART TRANSPLANT OR IMPLANT OF HEART ASSIST SY...          001   
4  HEART TRANSPLANT OR IMPLANT OF HEART ASSIST SY...          002   

  insurance_provider            insurance_plan  gross_standard_charges  \
0              Aetna                     Aetna               553176.28   
1  United Healthcare     United Healthcare HMO               635848.50   
2  United Healthcare  United Healthcare Option               835367.16   
3  United Healthcare     United Healthcare PPO               635848.50   
4              Aetna                     Aetna               333274.32   

   max_standard_charges  min_standard_charges   hospital_name  
0             835367.16             553176.28  USMD_Arlington  
1           

In [6]:
import pandas as pd

# Assuming 'pandas_df' is your Pandas DataFrame
# Export the DataFrame to a CSV file with a pipe delimiter
pandas_df.to_csv("C:\\Users\\tarun\\Documents\\Semester 4\\Analytics Practicum\\POC\\Project\\merged_data.csv", sep='\t', index=False)

print("Pandas DataFrame has been exported to 'merged_data.csv' with a pipe delimiter.")

Pandas DataFrame has been exported to 'merged_data.csv' with a pipe delimiter.


In [ ]:
# future scope : location, hospital name, branch_name, zipcode, latitude, longitude, city , state, phone, website, google maps link

: 